# Power Quality Classification using CNN

This notebook focusses on developing a Convolutional Neural Network which classifies a particular power signal into its respective power quality condition. The dataset used here contains signals which belong to one of the 5 classes(power quality condition). This means that each signal is characterized by 128 data points. Here the signals provided are in time domain.

The power quality condition with respect to the output class value is as follows: <br>
1 - Normal<br>
2 - 3rd harmonic wave<br>
3 - 5th harmonic wave<br>
4 - Voltage dip<br>
5 - transient<br>

In [ ]:
#importing the required libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
from scipy.fft import fft,fftfreq
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
#loading the dataset using pandas
data = pd.read_csv("../input/powerqualitydistributiondataset1/PowerQualityDistributionDataset1.csv")

In [ ]:
#The dataset is already preprocessed
data.drop(data.columns[[0]],axis=1,inplace=True)
data.shape

In [ ]:
data.head()

In [ ]:
data_arr = data.to_numpy()

### Data transformation

The data transformation steps employed here are as follows:<br>

1) Fourier Transform<br>
2) Normalization


In [ ]:
#In this segment we are plotting one wave from each class after applying fourier transformation 
yf = np.abs(fft(data_arr[0][0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, yf)
plt.show()
print("class",data_arr[0][128], "Normal wave")

yf = np.abs(fft(data_arr[1][0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, yf)
plt.show()
print("class",data_arr[1][128], "3rd harmonic wave")

yf = np.abs(fft(data_arr[3][0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, yf)
plt.show()
print("class",data_arr[3][128], "5th harmonic wave")

yf = np.abs(fft(data_arr[6][0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, yf)
plt.show()
print("class",data_arr[6][128], "Voltage dip")

yf = np.abs(fft(data_arr[8][0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, yf)
plt.show()
print("class",data_arr[8][128], "Transient wave")

In [ ]:
#here we are splitting the dataset in the ratio of 60%,20%,20% (training set,validation set, test set)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.loc[:,data.columns != 'output'],data['output'],test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [ ]:
#here we are overwritting the dataframe with the waves which we obtained after doing fourier transformation
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_val = x_val.to_numpy()

In [ ]:
#here we are performing normalization
transform = StandardScaler()
x_train_tr = transform.fit_transform(x_train)
x_test_tr = transform.fit_transform(x_test)
x_val_tr = transform.fit_transform(x_val)

## Model creation and training

In [ ]:
# get_dummies function is used here to perform one hot encoding of the y_* numpy arrays
y_train_hot = pd.get_dummies(y_train)
y_test_hot = pd.get_dummies(y_test)
y_val_hot = pd.get_dummies(y_val)

In [ ]:
print("Training",x_train.shape)
print(y_train_hot.shape)
print("Validation",x_val.shape)
print(y_val_hot.shape)
print("Test",x_test.shape)
print(y_test_hot.shape)

In [ ]:
#Reshaping the Data so that it could be used in 1D CNN
x_train_re = x_train.reshape(x_train_tr.shape[0],x_train_tr.shape[1], 1)
x_test_re = x_test.reshape(x_test_tr.shape[0],x_test_tr.shape[1], 1)
x_val_re = x_val.reshape(x_val_tr.shape[0],x_val_tr.shape[1], 1)

In [ ]:
x_train_re.shape

In [ ]:
#importing required modules for working with CNN
import tensorflow as tf
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2

In [ ]:
#initializing required parameters for the model
batch_size = 64
num_classes = 5
epochs = 20
input_shape=(x_train.shape[1], 1)

In [ ]:
model = Sequential()

model.add(Conv1D(128, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))

model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
#compiling the model

log_dir = "logs1/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
#training the model
history = model.fit(x_train_re, y_train_hot, batch_size=batch_size, epochs=epochs, validation_data=(x_val_re, y_val_hot), callbacks=[tensorboard_callback])


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs1/fit

In [ ]:
print(model.metrics_names)

## Model evaluation

In [ ]:
np.mean(history.history['accuracy']) 

In [ ]:
pred_acc = model.evaluate(x_test_re,y_test_hot)
print("Test accuracy is {}".format(pred_acc))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn

In [ ]:
array = confusion_matrix(y_test_hot.to_numpy().argmax(axis=1), model.predict(x_test_re).argmax(axis=1))


In [ ]:
array

In [ ]:
to_cm = pd.DataFrame(array, index = [i for i in ["Type-1","Type-2","Type-3","Type-4","Type-5"]],
                  columns = [i for i in ["Type-1","Type-2","Type-3","Type-4","Type-5"]])
plt.figure(figsize = (13,9))
sn.heatmap(to_cm, annot=True)